Attempting to load the occurrence dataframe and make it smaller by retaining just the necessary columns.

In [9]:
import os
import pandas as pd
import sys
from collections import Counter

from dwca.read import DwCAReader
from dwca.darwincore.utils import qualname as qn

In [10]:
home_dir = os.path.dirname(os.getcwd())
data_dir = "/Users/lbokeria/Documents/projects/gbif-species-trainer-data/"

family_name = "Sesiidae"

dwca_file_path = os.path.join(data_dir,"dwca_files",family_name+".zip")
occurrence_file_path = os.path.join(data_dir,"dwca_files",family_name,"occurrence.txt")

In [11]:
with DwCAReader(dwca_file_path) as dwca:
    # Get the file descriptor
    datafile_descriptor = dwca.get_descriptor_for(occurrence_file_path)
    print(datafile_descriptor)
    
    

NotADataFile: /Users/lbokeria/Documents/projects/gbif-species-trainer-data/dwca_files/Sesiidae/occurrence.txt is not a data file

In [26]:
with DwCAReader(os.path.join(data_dir,"dwca_files",family_name+".zip")) as dwca:
    occ_df = dwca.pd_read("occurrence.txt", parse_dates=True, on_bad_lines="skip")
    media_df = dwca.pd_read("multimedia.txt", parse_dates=True, on_bad_lines="skip")    

/Users/lbokeria/miniforge3/envs/gbif-species-trainer-AMI-fork/lib/python3.9/site-packages/dwca/read.py:203: DtypeWarning: Columns (2,9,26,35,39,42,46,47,54,56,57,58,61,62,64,65,66,67,68,69,71,74,75,76,77,78,79,81,82,84,85,86,87,88,89,90,91,92,93,94,96,97,98,99,100,101,103,109,110,111,113,115,116,117,118,119,121,122,123,125,126,127,128,129,130,132,135,136,142,143,144,145,147,148,149,150,151,152,153,154,155,156,157,158,159,160,167,170,171,172,173,175,176,177,178,179,180,181,183,185,186,187,189,190,191,192,193,195,205,207,210,211,212,214,215,227,243,247) have mixed types. Specify dtype option on import or set low_memory=False.
  df = read_csv(self.absolute_temporary_path(relative_path), **kwargs)


In [27]:
print(sys.getsizeof(occ_df)/1024/1024)

981.4541158676147


In [28]:
# Remove the extra rows and compare the size
fields_to_keep = [
    "id",
    "decimalLatitude",
    "decimalLongitude",
    "order",
    "family",
    "genus",
    "species",
    "acceptedScientificName",
    "year",
    "month",
    "day",
    "datasetName",
    "taxonID",
    "acceptedTaxonKey",
    "lifeStage",
    "basisOfRecord",
]

occ_df_trunc = occ_df[fields_to_keep]

print(sys.getsizeof(occ_df)/1024/1024)
print(sys.getsizeof(occ_df_trunc)/1024/1024)

speedup = sys.getsizeof(occ_df)/sys.getsizeof(occ_df_trunc)

print("speedup is " + str(round(speedup)))

981.4541158676147
78.22808361053467
speedup is 13


In [38]:
occ_df_usecols["geodeticDatum"]

0         WGS84
1         WGS84
2         WGS84
3         WGS84
4         WGS84
          ...  
127573    WGS84
127574    WGS84
127575    WGS84
127576    WGS84
127577    WGS84
Name: geodeticDatum, Length: 127578, dtype: object

In [44]:
with DwCAReader(os.path.join(data_dir,"dwca_files",family_name+".zip")) as dwca:
    occ_df_usecols = dwca.pd_read("occurrence.txt", 
                                  parse_dates=True,
                                  on_bad_lines="skip",
                                  usecols = fields_to_keep)


/Users/lbokeria/miniforge3/envs/gbif-species-trainer-AMI-fork/lib/python3.9/site-packages/dwca/read.py:203: DtypeWarning: Columns (61,76,180) have mixed types. Specify dtype option on import or set low_memory=False.
  df = read_csv(self.absolute_temporary_path(relative_path), **kwargs)


In [45]:
sys.getsizeof(occ_df_usecols)/1024/1024

85.77149105072021

In [41]:
occ_df_usecols_trunk = occ_df_usecols[fields_to_keep]

In [32]:
sys.getsizeof(occ_df_usecols_trunk)/1024/1024

78.22808361053467

In [46]:
occ_df_usecols.columns.difference(occ_df_trunc.columns)

Index(['geodeticDatum'], dtype='object')

In [ ]:
# Now read in chunks

with DwCAReader(os.path.join(data_dir,"dwca_files",family_name+".zip")) as dwca:
    occ_df_smaller = []
    for chunk in dwca.pd_read("occurrence.txt", chunksize=10):
        print("read the chunk now")
        print(chunk.shape)
        # occ_df_smaller.append(chunk[fields_to_keep])
        
    
    
    

In [23]:
occ_df_trunc.columns

Index(['id', 'decimalLatitude', 'decimalLongitude', 'order', 'family', 'genus',
       'species', 'acceptedScientificName', 'year', 'month', 'day',
       'datasetName', 'taxonID', 'acceptedTaxonKey', 'lifeStage',
       'basisOfRecord'],
      dtype='object')

In [24]:
occ_df_usecols.columns

Index(['id', 'datasetName', 'basisOfRecord', 'lifeStage', 'year', 'month',
       'day', 'decimalLatitude', 'decimalLongitude', 'taxonID', 'order',
       'family', 'genus', 'acceptedTaxonKey', 'species',
       'acceptedScientificName', 'geodeticDatum'],
      dtype='object')